In [1]:
import pandas as pd 
data = pd.read_excel("dataset.xlsx", sheet_name="Run 1_ PV_battery_model")
data= data.drop(["time" , "Snfc"] , axis=1)
datax = data.drop(columns=['Snet'])
dfy1 = data['Snet']

In [2]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(datax)
dfx1=scaler.transform(datax)
dfx1 = pd.DataFrame(dfx1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfx1,dfy1,test_size=0.2, random_state=42) 

# Blue Whale Optimization

In [4]:
import numpy as np
import random
from sklearn.metrics import root_mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed

# Individual Whale
class Whale:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.fitness = float('inf')

# BWO Algorithm
def blue_whale_optimize(func, dim, bounds, n_whales=7, max_iter=20, a_decay=2, early_stop=5):
    whales = [Whale(dim, bounds) for _ in range(n_whales)]
    best_whale = min(whales, key=lambda w: func(w.position))
    best_loss = func(best_whale.position)
    no_improve = 0

    for iter in range(max_iter):
        a = 2 - iter * (2 / max_iter)  # Linearly decreasing a from 2 to 0

        scores = Parallel(n_jobs=-1)(delayed(func)(w.position) for w in whales)

        for i, whale in enumerate(whales):
            fitness = scores[i]

            if fitness < whale.fitness:
                whale.fitness = fitness

            if fitness < best_loss:
                best_loss = fitness
                best_whale = Whale(dim, bounds)
                best_whale.position = whale.position.copy()
                best_whale.fitness = fitness
                no_improve = 0
            else:
                no_improve += 1

            if no_improve >= early_stop:
                return best_whale.position, best_loss

        for whale in whales:
            r = random.random()
            A = 2 * a * r - a
            C = 2 * r

            if random.random() < 0.5:
                D = np.abs(C * best_whale.position - whale.position)
                whale.position = best_whale.position - A * D
            else:
                b = 1
                l = random.uniform(-1, 1)
                distance_to_best = np.abs(best_whale.position - whale.position)
                whale.position = distance_to_best * np.exp(b * l) * np.cos(2 * np.pi * l) + best_whale.position

            whale.position = np.clip(whale.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return best_whale.position, best_loss

# Regression + BWO wrapper
def regression_bwo(model_class, param_bounds, X_train, X_test, y_train, y_test, n_whales=7, max_iter=12):
    def model_fitness(params):
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return root_mean_squared_error(y_test, y_pred)  # RMSE

    bounds = list(param_bounds.values())
    best_params, best_loss = blue_whale_optimize(model_fitness, len(bounds), bounds, n_whales, max_iter)

    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Example Usage
if __name__ == "__main__":
    # Example dataset

    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 20)}),
        "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 20), "min_samples_split": (2, 10)}),
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"\nOptimizing {name} using Blue Whale Optimization...")
        best_params, best_loss = regression_bwo(model_class, param_bounds, X_train, X_test, y_train, y_test)
        print(f"Best Parameters: {best_params}, Best Loss (RMSE): {best_loss:.4f}")



Optimizing GradientBoosting using Blue Whale Optimization...
Best Parameters: {'n_estimators': 81, 'learning_rate': 0.19071380620781497}, Best Loss (RMSE): 1.3612

Optimizing XGBRegressor using Blue Whale Optimization...
Best Parameters: {'n_estimators': 84, 'learning_rate': 0.17830877471388534}, Best Loss (RMSE): 0.3436

Optimizing RandomForest using Blue Whale Optimization...
Best Parameters: {'n_estimators': 80, 'max_depth': 18}, Best Loss (RMSE): 0.2131

Optimizing DecisionTree using Blue Whale Optimization...
Best Parameters: {'max_depth': 15, 'min_samples_split': 2}, Best Loss (RMSE): 0.2639


# Ant Colony Optimization (ACO) 